In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
import pickle

In [2]:
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load InceptionV3 Model (Corrected)
base_model_inception = InceptionV3(weights='imagenet', include_top=False)
model_inception = Model(inputs=base_model_inception.input,
                        outputs=GlobalAveragePooling2D()(base_model_inception.output))

# Load ResNet50 Model (Corrected)
base_model_resnet = ResNet50(weights='imagenet', include_top=False)
model_resnet = Model(inputs=base_model_resnet.input,
                     outputs=GlobalAveragePooling2D()(base_model_resnet.output))

print(model_inception.output.shape)  # Should print (None, 2048)
print(model_resnet.output.shape)     # Should print (None, 2048)


(None, 2048)
(None, 2048)


In [ ]:
# Define image directory
import os
valid_folder = os.path.join("E:/Project/DATA/Flicker8k_Dataset/val")



# Get list of images
image_files = [f for f in os.listdir(valid_folder) if f.endswith(".jpg")]

# Store features
features_valid_inception = {}
features_valid_resnet = {}

def extract_features(image_path, model, preprocess_func):
    try:
        img = load_img(image_path, target_size=(299, 299) if model.name == "inception_v3" else (224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_func(img)
        features = model.predict(img)
        return features
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

for img_name in image_files:
    img_path = os.path.join(valid_folder, img_name)
    
    # Extract using InceptionV3
    inception_features = extract_features(img_path, model_inception, inception_preprocess)
    if inception_features is not None:
        features_valid_inception[img_name] = inception_features

    # Extract using ResNet50
    resnet_features = extract_features(img_path, model_resnet, resnet_preprocess)
    if resnet_features is not None:
        features_valid_resnet[img_name] = resnet_features

# Save features
with open("features_valid_inception.pkl", "wb") as f:
    pickle.dump(features_valid_inception, f)

with open("features_valid_resnet.pkl", "wb") as f:
    pickle.dump(features_valid_resnet, f)

print("Feature extraction completed. Features saved as features_valid_inception.pkl and features_valid_resnet.pkl")
